In [1]:
import pandas as pd
import numpy as np
import pickle as pkl
import os
import virtual_biopsy_utils as vbu
import integration_images_features_utils as image_utils
import ast
import delong
import shap

from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import roc_curve, auc, roc_auc_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV

### Load sentara

In [2]:
sen_data = vbu.load_sentara(path = '../pkls/sentara.pkl', overwrite=True)

shape: (2395, 7713)


### Load shared features between maccabi and sentara 

In [3]:
shared = pkl.load(open('../pkls/shared_features_mac_sen.pkl', 'rb'))

len(shared)

1842

### Load shap feats

In [4]:
shap_feats = pkl.load(open('../pkls/sentara_union_shap_RF.pkl', 'rb'))

shap_feats = list(set(shap_feats))

### Add calculated features to sentara

* Add BMI estimation

In [5]:
sen_data = vbu.add_bmi_estimation_sentara(df=sen_data)

* Add likelihood of obesity estimation

In [6]:
sen_data = vbu.add_likelihood_obesity_estimation_sentara(df=sen_data)

* Breast density estimation

In [7]:
sen_data = vbu.add_density_estimation_sentara(df = sen_data)

### Use shared features only

In [8]:
sen_data = sen_data[shared]

### Split data

In [9]:
x_train, y_train, x_val, y_val, x_test, y_test = vbu.split_sentara(sen_data, 
                                train_path = '../pkls/sentara_train.pkl',
                                val_path = '../pkls/sentara_val.pkl', 
                                test_path = '../pkls/sentara_test.pkl', overwrite = True)

Number of samples in train: 1685, val: 357 and test: 353


### Transform multilabel to single label - only on the train set

In [10]:
sen_data_train = x_train.combine_first(y_train)

cancers = [x for x in sen_data_train if 'outcome_cancer_type' in x]

temp = []
for _, row in sen_data_train.iterrows():
    
    cols_one = row[cancers].loc[lambda x:x==True].index
    for col_name in cols_one:
        temp_row = row.copy()
        temp_row[cancers] = 0
        temp_row[col_name] = 1
        temp.append(temp_row)
        
sen_data_train = pd.DataFrame(temp)
multiindex = sen_data_train.index.tolist()
sen_data_train.reset_index(inplace=True)
sen_data_train['patient_id'] = [item[1] for item in multiindex]
sen_data_train['study_date'] = [item[2] for item in multiindex]
sen_data_train.set_index(['patient_id', 'study_date'], inplace = True, append=True)
sen_data_train.drop('index', axis = 1, inplace=True)
print('New number of samples of training set: %d' %sen_data_train.shape[0])

# Redefine new training dataframes with the single label samples
y_train = sen_data_train[[x for x in sen_data_train.columns if x.startswith('outcome_cancer_')]]
x_train = sen_data_train.drop(columns=[x for x in sen_data_train.columns if x.startswith('outcome_')])

New number of samples of training set: 1729


### Add imaging features to train and val sets

* Predictions

In [11]:
pred_file_path = '../input_files/final_tal_predictons_without_annotation_train_and_val.csv'

pred = image_utils.compute_predictions_images_sentara(pred_file_path)
x_train = x_train.join(pred.set_index('study_id'), on='study_id')
x_val = x_val.join(pred.set_index('study_id'), on='study_id')

* Findings size

In [12]:
pred_file_path = '../input_files/final_tal_predictons_without_annotation_train_and_val.csv'

finding_size = image_utils.compute_findings_size(pred_file_path)
x_train = x_train.join(finding_size.set_index('study_id'), on='study_id')
x_val = x_val.join(finding_size.set_index('study_id'), on='study_id')

* Types: Calcification, breast assymetry, tumor, architectural distortion, axillary lymphadenopa

In [13]:
pred_file_path = '../input_files/final_tal_predictons_without_annotation_train_and_val.csv'

types = image_utils.get_types(pred_file_path)
x_train = x_train.join(types.set_index('study_id'), on='study_id')
x_val = x_val.join(types.set_index('study_id'), on='study_id')

* Microcalcifications and Macrocalcifications

In [14]:
pred_file_path = '../input_files/final_tal_predictons_without_annotation_train_and_val.csv'

types = image_utils.get_micro_and_macro(pred_file_path)
x_train = x_train.join(types.set_index('study_id'), on='study_id')
x_val = x_val.join(types.set_index('study_id'), on='study_id')

In [15]:
# # adjust dtype to make xgboost run

# x_train['LCC_micro'] = x_train['LCC_micro'].astype('int') 
# x_train['RCC_micro'] = x_train['RCC_micro'].astype('int')
# x_train['LMLO_micro'] = x_train['LMLO_micro'].astype('int')
# x_train['RMLO_micro'] = x_train['RMLO_micro'].astype('int')

# x_val['LCC_micro'] = x_val['LCC_micro'].astype('int')
# x_val['RCC_micro'] = x_val['RCC_micro'].astype('int')
# x_val['LMLO_micro'] = x_val['LMLO_micro'].astype('int')
# x_val['RMLO_micro'] = x_val['RMLO_micro'].astype('int')

* Ella's features

In [16]:
spic_lesions_studies = [x.split('\t') for x in open('../input_files/additional_features_spiculated_lesions_sentara.txt').readlines()]
arch_dist_studies = [x.split('\t') for x in open('../input_files/additional_features_architectural_distortion_sentara.txt').readlines()]
susp_calc_studies = [x.split('\t') for x in open('../input_files/additional_features_suspicius_calcifications_sentara.txt').readlines()]

spic_lesions_studies = [item[1][:-1] for item in spic_lesions_studies[1:]]
arch_dist_studies = [item[1][:-1] for item in arch_dist_studies[1:]]
susp_calc_studies = [item[1][:-1] for item in susp_calc_studies[1:]]

x_train['spiculated_lesions_report'] = np.array([x in spic_lesions_studies for x in x_train.study_id.tolist()]).astype(int)
x_train['architectural_distortion_report'] = np.array([x in arch_dist_studies for x in x_train.study_id.tolist()]).astype(int)
x_train['suspicious_calcifications_report'] = np.array([x in susp_calc_studies for x in x_train.study_id.tolist()]).astype(int)

x_val['spiculated_lesions_report'] = np.array([x in spic_lesions_studies for x in x_val.study_id.tolist()]).astype(int)
x_val['architectural_distortion_report'] = np.array([x in arch_dist_studies for x in x_val.study_id.tolist()]).astype(int)
x_val['suspicious_calcifications_report'] = np.array([x in susp_calc_studies for x in x_val.study_id.tolist()]).astype(int)

### Drop features we don't want to use

In [17]:
# if large clinical only or large clinical+ small set images:
# x_train.drop(columns=['study_id'], inplace=True) #studyid
# x_val.drop(columns=['study_id'], inplace=True)

#if images only (small set):

# imaging_feats = [x for x in x_train if 'pred' in x]
# x_train = x_train[imaging_feats]
# x_val = x_val[imaging_feats]

# shap clinical + small set images

# imaging_feats = [x for x in x_train if 'pred' in x]
# x_train = x_train [imaging_feats + shap_feats]
# x_val = x_val [imaging_feats + shap_feats]

# shap clinical + full set images

image_full_set = [x for x in x_train if 'report' in x] + [x for x in x_train if 'calcification_in' in x] + \
[x for x in x_train if 'findings' in x] + [x for x in x_train if 'pred' in x] + \
['Calcification', 'Breast Assymetry', 'Tumor', 'Architectural Distortion', 'Axillary lymphadenopathy']
x_train = x_train [image_full_set + shap_feats]
x_val = x_val [image_full_set + shap_feats]

# if images full set only

# image_full_set = [x for x in x_train if 'report' in x] + [x for x in x_train if 'calcification_in' in x] + \
# [x for x in x_train if 'findings' in x] + [x for x in x_train if 'pred' in x] + \
# ['Calcification', 'Breast Assymetry', 'Tumor', 'Architectural Distortion', 'Axillary lymphadenopathy']
# x_train = x_train [image_full_set ]
# x_val = x_val [image_full_set ]


#### Fill missing values of categorical data with most frequence value

In [18]:
cat_feats = [x for x in x_train.columns if 'ind' in x] + [x for x in x_train.columns if 'cnt' in x] 
# +\
#               [ 'breast_density_past'] #we use this line if were running the complete clinical data 

x_train[cat_feats] = x_train[cat_feats].fillna(x_train[cat_feats].mode().iloc[0])
x_val[cat_feats] = x_val[cat_feats].fillna(x_val[cat_feats].mode().iloc[0])

#### Random Forest

In [19]:
classes = ['outcome_cancer_type_DCIS', 'outcome_cancer_type_Invasive', 'outcome_cancer_type_BenignHR',
           'outcome_cancer_type_Papilloma', 'outcome_cancer_type_Benign']

# RanFor_pipeline = Pipeline([
#     ('imputation', SimpleImputer(missing_values = np.nan, strategy = 'mean')), # impute continuous values with mean
#     ('scaler', MinMaxScaler()), 
#     ('clf', OneVsRestClassifier(RandomForestClassifier(random_state=42, n_estimators = 100), n_jobs=-1)),
# ])

RanFor_pipeline = Pipeline([
    ('imputation', SimpleImputer(missing_values = np.nan, strategy = 'mean')), # impute continuous values with mean
    ('scaler', MinMaxScaler()), 
    ('clf', RandomForestClassifier(random_state=42, n_estimators = 100))
])

# Parameters for grid
n_estimators = [50, 100, 200, 300]
max_features = ['auto', 'sqrt']
max_depth = [3, 4, 5, 6, 7, 8, 10]
min_samples_split = [2, 5, 10]


# grid_params_RF = {'clf__estimator__n_estimators': n_estimators,
#                  'clf__estimator__max_features': max_features,
#                  'clf__estimator__max_depth': max_depth,
#                  'clf__estimator__min_samples_split': min_samples_split }


grid_params_RF = {'clf__n_estimators': n_estimators,
                 'clf__max_features': max_features,
                 'clf__max_depth': max_depth,
                 'clf__min_samples_split': min_samples_split }

RF = RandomizedSearchCV(estimator = RanFor_pipeline,
                 param_distributions = grid_params_RF,
                 n_iter = 50, random_state=42, 
                 n_jobs = -1, scoring = 'roc_auc', cv=10)

predict_probs = []

for category in classes:
    print('**Processing class {} ...**'.format(category))
    
    if os.path.isfile('../pkls/cancer_prediction_shared_features_pkls/RandomForest/feature_set_both_shap_and_annotation/best_model_randomized_search_' + str(category) + '.pkl'):
        best_model = pkl.load(open('../pkls/cancer_prediction_shared_features_pkls/RandomForest/feature_set_both_shap_and_annotation/best_model_randomized_search_' + str(category) + '.pkl', 'rb'))
    else:

        RF.fit(x_train, y_train[category])
        y_pred = RF.predict(x_val)

        prob = RF.predict_proba(x_val)[:, 1]
        
        predict_probs.append(prob)

        print('AUC is {:.2f} [{:.2f}, {:.2f}]'.format(roc_auc_score(y_val[category], 
                                        prob), *delong.get_delong_ci(prob, y_val[category])))

        print('\n')

        pkl.dump(RF.best_estimator_, open('../pkls/cancer_prediction_shared_features_pkls/RandomForest/feature_set_both_shap_and_annotation/best_model_randomized_search_' + str(category) + '.pkl', 'wb')) 
    
    
# SHAP
    
#     if os.path.isfile('../pkls/cancer_prediction_shared_features_pkls/RandomForest/feature_set_clinical_only/shap_values_' + str(category) + '.pkl'):
#         shap_values = pkl.load(open('../pkls/cancer_prediction_shared_features_pkls/RandomForest/feature_set_clinical_only/shap_values_' + str(category) + '.pkl', 'rb'))
#     else:
#         model = RF.best_estimator_.named_steps['clf']
#         explainer = shap.TreeExplainer(model)
#         shap_values = explainer.shap_values(x_val)
#         pkl.dump(shap_values, open('../pkls/cancer_prediction_shared_features_pkls/RandomForest/feature_set_clinical_only/shap_values_' + str(category) + '.pkl', 'wb'))

**Processing class outcome_cancer_type_DCIS ...**
AUC is 0.69 [0.60, 0.78]


**Processing class outcome_cancer_type_Invasive ...**
AUC is 0.83 [0.78, 0.88]


**Processing class outcome_cancer_type_BenignHR ...**
AUC is 0.71 [0.54, 0.87]


**Processing class outcome_cancer_type_Papilloma ...**
AUC is 0.63 [0.50, 0.75]


**Processing class outcome_cancer_type_Benign ...**
AUC is 0.74 [0.68, 0.78]




In [20]:
np.savetxt('predict_probs_RF_feature_set_both_shap_feats_large_set_images.csv', predict_probs, delimiter=',')

In [ ]:
dcis = pkl.load(open('../pkls/cancer_prediction_shared_features_pkls/xgboost/feature_set_clinical_only/shap_values_outcome_cancer_type_DCIS.pkl', 'rb'))
shap.summary_plot(dcis, x_val.rename(columns = {'age':'Age'}))
# shap.plots.beeswarm(dcis, max_display=20, order=dcis.abs.mean(0), plot_size=(8,10))

In [ ]:
invasive = pkl.load(open('../pkls/cancer_prediction_shared_features_pkls/xgboost/feature_set_clinical_only/shap_values_outcome_cancer_type_Invasive.pkl', 'rb'))
shap.summary_plot(invasive, x_val.rename(columns = {'age':'Age'}))


In [ ]:
benignhr = pkl.load(open('../pkls/cancer_prediction_shared_features_pkls/xgboost/feature_set_clinical_only/shap_values_outcome_cancer_type_BenignHR.pkl', 'rb'))
papilloma = pkl.load(open('../pkls/cancer_prediction_shared_features_pkls/xgboost/feature_set_clinical_only/shap_values_outcome_cancer_type_Papilloma.pkl', 'rb'))
benign = pkl.load(open('../pkls/cancer_prediction_shared_features_pkls/xgboost/feature_set_clinical_only/shap_values_outcome_cancer_type_Papilloma.pkl', 'rb'))

In [ ]:
# union_shap = x_val.columns[np.argsort(np.abs(benign).mean(0))].tolist()[::-1][:20] +\
#         x_val.columns[np.argsort(np.abs(dcis).mean(0))].tolist()[::-1][:20] +\
#         x_val.columns[np.argsort(np.abs(benignhr).mean(0))].tolist()[::-1][:20] +\
#         x_val.columns[np.argsort(np.abs(papilloma).mean(0))].tolist()[::-1][:20] +\
#         x_val.columns[np.argsort(np.abs(invasive).mean(0))].tolist()[::-1][:20]
        
# print(len(list(set(union_shap))))

# with open('../pkls/sentara_union_shap_RF.pkl', 'wb') as handle:
#     pkl.dump(union_shap, handle, protocol=pkl.HIGHEST_PROTOCOL)